In [3]:
import random
import numpy as np

In [4]:
class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit = 4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        
        pit -= 1

        if self.current_player == 1:
            valid_range = range(self.p1_pits_index[0], self.p1_pits_index[1] + 1)
        else:
            valid_range = range(self.p2_pits_index[0], self.p2_pits_index[1] + 1)

        pit_index = list(valid_range)[pit] if pit < len(valid_range) else -1

        if pit_index not in valid_range:
            return False
        if self.board[pit_index] == 0:
            return False

        return True
        
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        valid_pits = []
        if self.current_player == 1:
            start, end = self.p1_pits_index
        else:
            start, end = self.p2_pits_index

        for i in range(start, end + 1):
            if self.board[i] > 0:
                valid_pits.append(i - start + 1)

        if not valid_pits:
            return None
        return random.choice(valid_pits)
        


    
    def play(self, pit):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        """
        
        if not self.valid_move(pit):
            print("INVALID MOVE")
            return self.board

        if self.winning_eval():
            print("GAME OVER")
            return self.board

        pit -= 1
        if self.current_player == 1:
            pit_index = self.p1_pits_index[0] + pit
        else:
            pit_index = self.p2_pits_index[0] + pit

        stones = self.board[pit_index]
        self.board[pit_index] = 0
        index = pit_index

        while stones > 0:
            index = (index + 1) % len(self.board)

            if self.current_player == 1 and index == self.p2_mancala_index:
                continue
            if self.current_player == 2 and index == self.p1_mancala_index:
                continue
            self.board[index] += 1
            stones -= 1

        if self.current_player == 1:
            if index in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1) and self.board[index] == 1:
                opposite_index = self.p2_pits_index[1] - (index - self.p1_pits_index[0])
                captured = self.board[opposite_index]
                if captured > 0:
                    self.board[self.p1_mancala_index] += captured + 1
                    self.board[index] = 0
                    self.board[opposite_index] = 0
        else:
            if index in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1) and self.board[index] == 1:
                opposite_index = self.p1_pits_index[1] - (index - self.p2_pits_index[0])
                captured = self.board[opposite_index]
                if captured > 0:
                    self.board[self.p2_mancala_index] += captured + 1
                    self.board[index] = 0
                    self.board[opposite_index] = 0

        self.moves.append((self.current_player, pit + 1))

        if (self.current_player == 1 and index != self.p1_mancala_index) or (self.current_player == 2 and index != self.p2_mancala_index):
            self.current_player = 2 if self.current_player == 1 else 1
                
        
        return self.board
    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        
        
        p1_side_empty = all(stone == 0 for stone in self.board[self.p1_pits_index[0]:self.p1_pits_index[1] + 1])
        p2_side_empty = all(stone == 0 for stone in self.board[self.p2_pits_index[0]:self.p2_pits_index[1] + 1])

        if p1_side_empty or p2_side_empty:
            self.board[self.p1_mancala_index] += sum(self.board[self.p1_pits_index[0]:self.p1_pits_index[1] + 1])
            self.board[self.p2_mancala_index] += sum(self.board[self.p2_pits_index[0]:self.p2_pits_index[1] + 1])
            for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1):
                self.board[i] = 0
            for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1):
                self.board[i] = 0


            if self.board[self.p1_mancala_index] > self.board[self.p2_mancala_index]:
                print("Player 1 wins!")
            elif self.board[self.p1_mancala_index] < self.board[self.p2_mancala_index]:
                print("Player 2 wins!")
            else:
                print("It’s a tie!")
            return True
        return False

In [5]:
p1_wins = 0
p2_wins = 0
ties = 0
moves_to_end = []
for match in range(100):
    moves = 0

    game = Mancala(pits_per_player=6, stones_per_pit=4)
    while not game.winning_eval():
        pit = game.random_move_generator()
        if pit is None:
            break

        game.play(pit)
        moves += 1
    if game.board[game.p1_mancala_index] > game.board[game.p2_mancala_index]:
        p1_wins += 1
    elif game.board[game.p1_mancala_index] < game.board[game.p2_mancala_index]:
        p2_wins += 1
    else:
        ties += 1
    moves_to_end += [moves]
print("p1 wins: ",p1_wins)
print("p2 wins: ",p2_wins)
print("ties: ",ties)
print("average moves: ", np.mean(moves_to_end))

        

Player 1 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 1 wins!
Player 2 wins!
Player 1 wins!
Player 2 wins!
Player 1 wins!
Player 2 wins!
Player 1 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 2 wins!
Player 1 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 2 wins!
Player 1 wins!
Player 2 wins!
Player 1 wins!
Player 1 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 2 wins!
Player 1 wins!
Player 2 wins!
Player 1 wins!
Player 1 wins!
Player 2 wins!
Player 1 wins!
Player 2 wins!
Player 1 wins!
It’s a tie!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 2 wins!
Player 1 wins